In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import quote
import requests
import re          #정규표현식을 위해
from tqdm.notebook import tqdm

In [ ]:
#https://www.hanbit.co.kr/store/books/new_book_list.html
#제목,저자,출간일,쪽수,가격,역자

In [ ]:
base_url='https://www.hanbit.co.kr/'
sub_url='store/books/new_book_list.html'
url = base_url+sub_url
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')

In [66]:
lis = soup.select('.sub_book_list')
len(lis)

20

In [ ]:
li = lis[0]
href=li.find('a')['href']
href

'/store/books/look.php?p_code=B1207366943'

In [ ]:
li

* sub page에서 정보를 가져옴

In [72]:
res=requests.get(base_url+href)
book_soup=BeautifulSoup(res.text,'html.parser')
book_soup

<!DOCTYPE html>

<html lang="ko">
<head>
<!--[if lte IE 8]>
<script>
  location.replace('/support/explorer_upgrade.html');
</script>
<![endif]-->
<meta charset="utf-8"/>
<title>엄마가 만드는 초등 수학 자신감</title>
<link href="https://www.hanbit.co.kr/images/common/hanbit.ico" rel="shortcut icon"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="website" property="og:type"/>
<meta content="엄마가 만드는 초등 수학 자신감" property="og:title"/>
<meta content="이 책은 미취학부터 중등까지, 가정에서 할 수 있는 수학 공부 습관 잡기와 학년별 수학 공부 방법을 상세히 안내합니다." property="og:description"/>
<meta content="https://www.hanbit.co.kr/data/books/B5658376953_m.jpg" property="og:image"/>
<meta content="https://www.hanbit.co.kr//store/books/look.php?p_code=B5658376953" property="og:url"/>
<link href="https://www.hanbit.co.kr//store/books/look.php?p_code=B5658376953" rel="canonical">
<meta content="초등수학ㅊ수학공부법,엄마표수학,공부잘하는아이,초등공부법,공부습관" name="keywords"/>
<meta content="이 책은 미취학부터 중등까지, 가정에서 할 수 있는 수학 공부 습관 잡기와 학년별 수학 공부 방법을 상세히 안내합니다." na

* 데이터 가져오기

In [ ]:
info=book_soup.select_one('.store_product_info_box')
title = info.find('h3').string
title

'구글 엔지니어는 이렇게 일한다'

In [ ]:
book_list=info.select('.info_list > li')
author = book_list[0].find('span').get_text().strip()
author

'타이터스 윈터스 , 톰 맨쉬렉 , 하이럼 라이트'

In [ ]:
if len(book_list) >= 5:
  translator = book_list[1].find('span').get_text().strip().split(',')[0]
else:
  translator = book_list[1].find('span').get_text().strip()

translator

'개앞맵시(이복연)'

In [ ]:
date = book_list[-4].find('span').get_text().strip()
date

'2022-05-10'

In [ ]:
page = book_list[-3].find('span').get_text().strip().split()[0]
page

'704'

In [ ]:
payments = book_soup.select_one('.payment_box.curr').find_all('p')
sales = payments[1].select_one('.pbr').get_text().split('(')[0]
sales = int(re.sub('[^0-9]','',sales))
sales

40500

In [ ]:
#'제목','저자','번역','출시일',

In [ ]:
line=[]
for li in lis:
  list = li
  href=list.find('a')['href']
  res=requests.get(base_url+href)
  book_soup=BeautifulSoup(res.text,'html.parser')
  info=book_soup.select_one('.store_product_info_box')
  book_list=info.select('.info_list > li')
  title = info.find('h3').string
  author = book_list[0].find('span').get_text().strip()
  translator = book_list[1].find('span').get_text().strip()
  date = book_list[-4].find('span').get_text().strip()
  page = book_list[-3].find('span').get_text().strip().split()[0]
  payments = book_soup.select_one('.payment_box.curr').find_all('p')
  sales = payments[1].select_one('.pbr').get_text().split('(')[0]
  sales = int(re.sub('[^0-9]','',sales))
  line.append({'제목':title,'저자':author,'번역':translator,'출시일':date,'쪽수':page,'가격':sales})
line

In [ ]:
df=pd.DataFrame(line)
df

,제목,저자,번역,출시일,쪽수,가격
0,구글 엔지니어는 이렇게 일한다,"타이터스 윈터스 , 톰 맨쉬렉 , 하이럼 라이트",개앞맵시(이복연),2022-05-10,704,40500
1,실무자를 위한 그래프 데이터 활용법,"데니즈 고즈넬 , 마티아스 브뢰헬러",우정은,2022-05-10,448,33300
2,완성된 웹사이트로 배우는 HTML&CSS 웹 디자인,Mana,신은화,2022-05-10,360,19800
3,리얼 국내여행 2022~2023(개정판),배나영,2022-05-03,436 쪽,9791190846400,16200
4,리얼 제주 [2022~2023년 최신판],"김태연(제주레이) , 양정임(제주규리)",2022-05-02,2022-05-02,352,15300
5,벌레가 되어도 출근은 해야 해,박윤진,2022-05-02,2022-05-02,312,14400
6,"일잘러의 비밀, 엑셀 대신 파이썬으로 업무 자동화하기",포스코인재창조원,2022-05-02,2022-05-02,368,21600
7,똑똑한 두뇌 연습 : 300개 미로찾기,김정겸,2022-04-30,2022-04-30,280,13320
8,MLOps 도입 가이드,데이터이쿠,동동구,2022-04-29,200,19800
9,"엑셀, R, 파이썬으로 시작하는 데이터 분석",조지 마운트,시진,2022-04-29,280,19800


In [ ]:
lines = []
for page in tqdm(range(1,11)):
  url=f'{base_url}{sub_url}?page={page}'
  res=requests.get(url)
  soup=BeautifulSoup(res.text,'html.parser')
  lis=soup.select('.sub_book_list')
  for li in lis:
      book_url = li.find('a')['href']
      res = requests.get(base_url + book_url)
      book_soup = BeautifulSoup(res.text, 'html.parser')

      info = book_soup.select_one('.store_product_info_box')
      title = info.find('h3').string
      book_lis = info.select('.info_list > li')
      author = book_lis[0].find('span').get_text().strip()
      if len(book_lis) == 5:
          translator = '-'
      else:
          translator = book_lis[1].find('span').get_text().strip()
      date = book_lis[-4].find('span').get_text().strip()
      page = book_lis[-3].find('span').get_text().strip().split()[0]
      
      payments = book_soup.select_one('.payment_box.curr').find_all('p')
      sales = payments[1].select_one('.pbr').get_text().split('(')[0]
      sales = int(re.sub('[^0-9]','',sales))

      lines.append([title,author,translator,date,page,sales])

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','페이지','판매가'])
df.tail()

,제목,저자,번역,출간일,페이지,판매가
195,재미있고 빠른 숫자 쓰기 1권 : 0부터 50까지,한빛학습연구회,-,2021-07-26,88,6750
196,재미있고 빠른 숫자 쓰기 2권 : 0부터 100까지,한빛학습연구회,-,2021-07-26,96,6750
197,재미있고 빠른 알파벳 쓰기,한빛학습연구회,-,2021-07-26,88,6750
198,비즈니스 애널리틱스,"Sanjiv Jaggia , Alison Kelly , Kevin Lertwacha...","장혜정 , 이상규 , 오세환 , 변종복",2021-07-20,704,38000
199,이기적이라 살아남았습니다,이마이즈미 다다아키,"김지연 , 이정모(감수)",2021-07-20,176,12420


In [ ]:
df.to_csv('한빛신규도서.csv',index=False)